In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

import brighteyes_ism.dataio.mcs as mcs
import brighteyes_ism.analysis.Graph_lib as gr
import brighteyes_ism.simulation.PSF_sim as sim

from s2ism import s2ism as amd
import s2ism.psf_estimator as est



from tqdm import trange

C:\Users\ggarre\AppData\Local\anaconda3\envs\s2ism\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

plt.close('all')

Importing the raw ISM dataset from the bunch of experimental datasets shown in the paper. 
The experimental dataset can be downloaded from https://doi.org/10.5281/zenodo.11284051.

Here, we download and decompress the data into the Downldoas folder.

In [3]:
import requests
from tqdm import tqdm, trange
from pathlib import Path
import os
import zipfile

def download(url, fname):
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
            desc=fname,
            total=total,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
            
def extract(filename):
    dir_name = filename[:-4]
    os.mkdir(dir_name)
    with zipfile.ZipFile(filename, 'r') as zf:
        for member in tqdm(zf.infolist(), desc='Extracting '):
            try:
                zf.extract(member, dir_name)
            except zipfile.error as e:
                pass

In [4]:
downloads_path = str(Path.home() / 'Downloads')

url_data = 'https://zenodo.org/records/11284051/files/h5_files_s2ism.zip'
name_data = 'h5_files_s2ism.zip'

filename = os.path.join(downloads_path, name_data)

In [5]:
if not os.path.isfile(filename):
    print('Downloading data:' + filename + '\n')
    download(url_data, filename)
else:
    print('File already downloaded.')

C:\Users\ggarre\Downloads\h5_files_s2ism.zip: 100%|██████████| 3.65G/3.65G [11:57<00:00, 5.47MiB/s]


In [7]:
dir_name = filename[:-4]

if not os.path.isfile(dir_name):
    print('Extracting compressed data:\n')
    extract(filename)

Extracting compressed data:



FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\ggarre\\Downloads\\h5_files_s2ism'

In [ ]:
file = 'Mithocondria_timelapse_data-09-11-2023-12-47-10.h5'

fullpath = os.path.join(dir_name, file)

data, meta = mcs.load(fullpath)

In [ ]:
print(data.shape)

Squeezing the unnecessary dimensions and summing over the temporal bins to obtain a good SNR, nedded to launch the analysis: 

In [ ]:
dset = np.squeeze(data)
del data

In [ ]:
dset = dset.sum(axis = -2)

Exctracting some dimensions from the raw dataset needed for the upcoming analysis:

In [ ]:
sz = dset.shape
N = int( np.sqrt(sz[-1]) )

Simulating the complete PSFs extracting some fundamental characteristics of the imaging system from the raw dataset (for a deeper description of this process, check the 'Reconstruction_step_by_step_exp.ipynb' file notebook attached in the GitHub part of the ope-source code). In such a way we maximize the physical reliability of the simulation, obtaining PSFs as close as possible to the experimental ones (effectively describing the custom ISM setup used to acquire this particular dataset):

In [ ]:
exPar = sim.simSettings()
exPar.wl = 561
exPar.mask_sampl = 101

emPar = exPar.copy()
emPar.wl = 590

Nx = meta.nx
Ny = meta.ny

grid = sim.GridParameters()
grid.pxsizex = meta.dx*1e3
grid.Nz = 2

psf_fin, det_psf, exc_psf = est.psf_estimator_from_data(dset, exPar, emPar, grid, z_out_of_focus='300')   
#out-of-focus position for the reconstruction is a-priori posed, and based on quantitative analysis deeply described in the paper itself.


In [ ]:
spad_size = grid.spad_size() / emPar.airy_unit
print(f'SPAD array size = {spad_size:.1f} AU')

Qualitative check of the simulated PSFs:

In [ ]:
gr.ShowDataset(psf_fin[0])
gr.ShowDataset(psf_fin[1])

In [ ]:
time_frames = sz[0]

ml_frames = np.empty( (time_frames, meta.nx, meta.ny) )

for f in trange(time_frames):
    s2ism_result = amd.max_likelihood_reconstruction(dset[f], psf_fin, stop = 'fixed', max_iter = 20, rep_to_save ='last')
    
    obj = amd_result[0]
    
    ml_frames[f] = obj[1]

Qualitative check of the s2ism in-focus reconstruction:

In [ ]:
gr.ShowStack(ml_frames, meta.dx, 1)